In [2]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import os 

# Import cleaned midwest data
file = os.path.join('MidwestMurderData.csv')

# Read into DataFrame
midwest_homicide_df = pd.read_csv(file)
midwest_homicide_df.head()

,index,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,189,197601001IA00801,"Boone, IA",IA00801,Iowa,Boone,Municipal police,FBI,No,1976,...,Unknown or not reported,Knife or cutting instrument,Relationship not determined,Other arguments,NaN,0,0,30180.0,Iowa,Rural Iowa
1,190,197601001IA04400,"Henry, IA",IA04400,Iowa,Henry County,Sheriff,FBI,Yes,1976,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Acquaintance,Robbery,NaN,0,0,30180.0,Iowa,Rural Iowa
2,191,197601001IA05202,"Johnson, IA",IA05202,Iowa,Iowa City,Municipal police,FBI,Yes,1976,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Acquaintance,Other arguments,NaN,0,0,30180.0,Iowa,"Iowa City, IA"
3,192,197601001IA05701,"Linn, IA",IA05701,Iowa,Cedar Rapids,Municipal police,FBI,Yes,1976,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Husband,Other arguments,NaN,0,0,30180.0,Iowa,"Cedar Rapids, IA"
4,193,197601001IA05703,"Linn, IA",IA05703,Iowa,Hiawatha,Municipal police,FBI,Yes,1976,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Stranger,Brawl due to influence of alcohol,NaN,0,0,30180.0,Iowa,"Cedar Rapids, IA"


In [3]:
midwest_homicide_df.columns

Index(['index', 'ID', 'CNTYFIPS', 'Ori', 'State', 'Agency', 'Agentype',
       'Source', 'Solved', 'Year', 'StateName', 'Month', 'Incident',
       'ActionType', 'Homicide', 'Situation', 'VicAge', 'VicSex', 'VicRace',
       'VicEthnic', 'OffAge', 'OffSex', 'OffRace', 'OffEthnic', 'Weapon',
       'Relationship', 'Circumstance', 'Subcircum', 'VicCount', 'OffCount',
       'FileDate', 'fstate', 'MSA'],
      dtype='object')

### Create data frame of total homicides per year.

In [7]:
midwest_total = midwest_homicide_df[['Year', 'ID']]
midwest_total_group = midwest_total.groupby(['Year']).count().reset_index()
midwest_total_group.head()

,Year,ID
0,1976,4209
1,1977,4081
2,1978,4019
3,1979,4567
4,1980,4670


### Preparing data for visualizations about homicide solve rates.

In [4]:
# Remove unneccessary columns
midwest_solved = midwest_homicide_df[['Year', 'Solved']]

# Group by year to get count of total homicides per year
midwest_year_group = midwest_solved.groupby(['Year']).count()

# Remove unsolved cases to get count of solved cases per year
solved = midwest_solved.loc[midwest_solved["Solved"] == "Yes"]
midwest_solved_group = solved.groupby(['Year']).count()

# Calculate solve rate
midwest_solved_group["% Solved"] = midwest_solved_group["Solved"] / midwest_year_group["Solved"] * 100
midwest_solved_group = midwest_solved_group.reset_index(drop=False)

midwest_solved_group.head()

,Year,Solved,% Solved
0,1976,3168,75.267284
1,1977,3059,74.957118
2,1978,3064,76.237870
3,1979,3394,74.315743
4,1980,3400,72.805139


In [5]:
# Choose three cities to look at 
midwest_cities = ['Chicago-Naperville-Joliet, IL-IN-WI',
                  'Indianapolis, IN',
                  'Minneapolis-St. Paul-Bloomington, MN-WI',]

# Remove unneccessary columns from original DataFrame
cities_df = midwest_homicide_df[['MSA', 'Year', 'Solved']]

# Pull out three specified cities
cities_df = cities_df[cities_df['MSA'].isin(midwest_cities)]
cities_df = cities_df.reset_index(drop=True)

# Group by city and year to get total count of homicide cases
cities_year_group = cities_df.groupby(['MSA', 'Year']).count().reset_index()

# Remove unsolved cases to get count of solved cases
cities_solved = cities_df.loc[cities_df["Solved"] == "Yes"]
cities_solved = cities_solved.groupby(['MSA', 'Year']).count().reset_index()

# Merge DataFrames
new_df = pd.merge(cities_solved, cities_year_group,  how='outer', left_on=['MSA','Year'], right_on = ['MSA','Year'])

# Rename columns
new_df = new_df.rename(columns={'MSA': 'City', 
                                'Solved_x': 'Homicides Solved',
                                'Solved_y': 'Total Homicides'})

# Calculate solve rates
new_df["% Solved"] = new_df["Homicides Solved"] / new_df["Total Homicides"] * 100
new_df.head()

,City,Year,Homicides Solved,Total Homicides,% Solved
0,"Chicago-Naperville-Joliet, IL-IN-WI",1976,850,1101,77.202543
1,"Chicago-Naperville-Joliet, IL-IN-WI",1977,825,1123,73.463936
2,"Chicago-Naperville-Joliet, IL-IN-WI",1978,829,1098,75.500911
3,"Chicago-Naperville-Joliet, IL-IN-WI",1979,911,1251,72.821743
4,"Chicago-Naperville-Joliet, IL-IN-WI",1980,887,1261,70.340999


In [6]:
# Create a DataFrame for each city
chicago = new_df[new_df["City"] == 'Chicago-Naperville-Joliet, IL-IN-WI']
indy = new_df[new_df["City"] == 'Indianapolis, IN']
mpls = new_df[new_df["City"] == 'Minneapolis-St. Paul-Bloomington, MN-WI']